In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)


In [83]:
sizeData = len(df.index)
trainSize = int(0.5*sizeData)
trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)


In [84]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["vraisemblance"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

spamProbality = spamTotal/hamTotal

C:\Users\Candidat\AppData\Local\Temp\ipykernel_15600\3630862678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFSpam["text"]=trainDFSpam["text"].str.lower()
C:\Users\Candidat\AppData\Local\Temp\ipykernel_15600\3630862678.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFHam["text"]=trainDFHam["text"].str.lower()


In [87]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)
print(TestPhraseList(["hello","call","to","win","your","prize"]))
print(TestPhraseList(["hello","call","me","i","miss","you"]))

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


print(TestPhrase("Hi how are you ? I have a wonderful proposition for you ! Gain your wonderful price by clicking on this link"))


608093.1851883662
0.00024645426884846715
3.919916687086737


In [88]:
testDF["text"]=testDF["text"].str.split()

In [89]:
testDF["result"]=testDF["text"].apply(TestPhraseList,spamProba=0.5)
testDF["Finalresult"]=['spam' if x > 1 else 'ham' for x in testDF['result']]
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
wrong/len(testDF.index)

0.04845656855707107